# Reading in Daily Precipitation Data 

This daily precipitation represents spatially gridded daily total precipitation at 4km grid cell resolution. Distribution of the point measurements to the spatial grid was accomplished using the PRISM model, developed and applied by Dr. Christopher Daly of the [PRISM Climate Group](https://prism.oregonstate.edu) at Oregon State University . 


## Workflow 

**Data Ingestion:**
* The code reads BIL (Band Interleaved by Line) raster files containing precipitation data
* It also loads zip code boundary data from a shapefile to map precipitation to geographic areas
* The data spans years 2023-2025

**Location Data Preprocessing:**
* For each zip code area, a representative point is calculated to ensure data sampling occurs within the polygon
* BIL files are filtered by year and processed in batches of 10 files at a time
* The code extracts the date information from each filename

**Precipitation Data Processing:**
* For each BIL file, the code:
  * Reads the raster data using the rasterio library
  * Extracts precipitation values at each zip code's representative point
  * Handles coordinate system transformations if needed

**Data Merging:**
* Results from each batch are joined together using common columns (geographic identifiers and coordinates)
* The data is periodically checkpointed to disk to optimize memory usage
* Each year's data is processed separately

**Output Storage:**
* Final datasets are written as Parquet files to intermediate storage locations
* The datasets include precipitation values mapped to zip codes with their coordinates
* Each year's data is saved separately ("/FileStore/intermediate_output/YYYY_precipitation_data")

<img src="assets/weather_data_processing_viz.png" alt="alt text" width="800" height="400">

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
import geopandas as gpd
import numpy as np
import gc
import os 
import re
import datetime
import rasterio
from rasterio.transform import rowcol
from functools import reduce
from pyspark.sql import DataFrame

In [0]:
# Initialize Spark Session
spark = SparkSession.builder.appName("Precipitation Data Processing").getOrCreate()

## Functions to Process Data 

In [0]:
def list_bil_files(base_path, recursive=True):
    """
    List all bile files in directory
    """
    all_files = []
    
    # Remove /dbfs prefix if it exists for listing
    list_path = base_path.replace("/dbfs", "") if base_path.startswith("/dbfs") else base_path
    
    if recursive:
        files = dbutils.fs.ls(list_path)
        for file_info in files:
            path = file_info.path
            if file_info.isDir():
                # Recursively list files in subdirectory
                all_files.extend(list_bil_files(f"{path}", recursive))
            elif path.lower().endswith('.bil'):
                # Add /dbfs prefix for local file system access
                # Convert dbfs:/ path to /dbfs/ path
                local_path = "/dbfs" + path[5:] if path.startswith("dbfs:") else f"/dbfs{path}"
                all_files.append(local_path)
    else:
        files = dbutils.fs.ls(list_path)
        for file_info in files:
            path = file_info.path
            if path.lower().endswith('.bil'):
                # Convert dbfs:/ path to /dbfs/ path
                local_path = "/dbfs" + path[5:] if path.startswith("dbfs:") else f"/dbfs{path}"
                all_files.append(local_path)
    
    return all_files

def load_zip_shapefile():
    """ 
    Load the zip code shapefile
    """
    zip_shapefile = os.path.join(data_path, "tl_2020_us_zcta520.shp")
    
    if os.path.exists(zip_shapefile):
        zip_gdf = gpd.read_file(zip_shapefile)
        print(f"Loaded {len(zip_gdf)} zip code boundaries")
        return zip_gdf
    else:
        print("Zip code shapefile not found. Will proceed without zip code assignment.")
        return None

def get_zip_points(zip_gdf):
    """
    Get representative points for each zip code polygon
    """
    points_data = []
    
    for idx, row in zip_gdf.iterrows():
        # Get a point guaranteed to be inside the polygon
        point = row.geometry.representative_point()
        
        # Create a dictionary with all attributes plus coordinates
        point_data = {col: row[col] for col in zip_gdf.columns if col != 'geometry'}
        point_data['longitude'] = point.x
        point_data['latitude'] = point.y
        point_data['geometry'] = point
        
        points_data.append(point_data)
    
    # Create a GeoDataFrame with the points
    points_gdf = gpd.GeoDataFrame(points_data, geometry='geometry', crs=zip_gdf.crs)
    
    return points_gdf

def read_bil_file(file_path):
    try:
        with rasterio.open(file_path) as src:
            # Read the raster data
            data = src.read(1)  # Read the first band
            
            # Get metadata
            metadata = {
                "width": src.width,
                "height": src.height,
                "crs": src.crs,
                "transform": src.transform,
                "bounds": src.bounds,
                "nodata": src.nodata
            }
            
            return data, metadata
    except Exception as e:
        print(f"Error reading {file_path}: {str(e)}")
        return None, None

def get_values_at_points(points_gdf, bil_data, bil_metadata):
    """
    Extract values from BIL data at specified points.
    """
    # Get the transformation from the metadata
    transform = bil_metadata["transform"]
    bil_crs = bil_metadata.get("crs")
    
    # If BIL has a CRS and it's different from points, reproject points
    if bil_crs is not None and str(bil_crs) != str(points_gdf.crs):
        points_gdf = points_gdf.to_crs(bil_crs)
    
    # Initialize results array
    values = []
    
    # Extract value at each point
    for idx, point in points_gdf.iterrows():
        # Get point coordinates
        x, y = point.geometry.x, point.geometry.y
        
        try:
            # Convert coordinates to pixel indices
            row, col = rowcol(transform, x, y)
            
            # Check if point is within raster bounds
            if (0 <= row < bil_metadata["height"] and 0 <= col < bil_metadata["width"]):
                value = bil_data[row, col]
                # Check for nodata value
                if bil_metadata["nodata"] is not None and value == bil_metadata["nodata"]:
                    values.append(np.nan)
                else:
                    values.append(float(value))
            else:
                values.append(np.nan)
        except Exception as e:
            print(f"Error getting value at point ({x}, {y}): {str(e)}")
            values.append(np.nan)
    
    return values


def map_bil_to_zip_codes(bil_files, points_gdf, sample_count=None):
    """
    Process BIL files and map values to zip codes using Spark
    """
    # If sample_count is provided, limit the number of files
    if sample_count is not None and sample_count < len(bil_files):
        bil_files = bil_files[:sample_count]
    
    # Initialize empty list for all results
    all_results = []
    
    # Process each BIL file
    for bil_file in bil_files:
        file_name = os.path.basename(bil_file)
        column_name = os.path.splitext(file_name)[0]
        
        print(f"Processing {file_name}...")
        
        # Read BIL file
        bil_data, bil_metadata = read_bil_file(bil_file)
        
        if bil_data is None or bil_metadata is None:
            print(f"Skipping {file_name} due to reading errors")
            continue
        
        # Extract values at zip code points
        values = get_values_at_points(points_gdf, bil_data, bil_metadata)
        
        # Add to results list
        for idx, point in points_gdf.iterrows():
            result = {col: point[col] for col in points_gdf.columns if col != 'geometry'}
            result['longitude'] = point.geometry.x
            result['latitude'] = point.geometry.y
            result['file_name'] = file_name
            result['column_name'] = column_name
            result['value'] = values[idx]
            all_results.append(result)
    
    # Convert to Spark DataFrame
    if all_results:
        result_df = spark.createDataFrame(all_results)
        
        # Pivot the data to have one column per BIL file
        result_pivot = result_df.groupBy([col for col in result_df.columns 
                                         if col not in ['file_name', 'column_name', 'value']])\
                                .pivot('column_name')\
                                .agg(F.first('value'))
        
        return result_pivot
    else:
        return spark.createDataFrame([])

def list_and_filter_bil_files(data_path, year_filter="2025"):
    """
    Get all BIL files at once and filter in one pass
    """
    all_files = list_bil_files(data_path)
    filtered_files = [f for f in all_files if year_filter in f]
    print(f"Found {len(filtered_files)} BIL files for {year_filter}")
    return filtered_files

def process_files_spark(bil_files, points_gdf, batch_size=10):
    """
    Parallelize batch processing with Spark
    """
    # Create schema for result to avoid schema inference
    result_schema = StructType([
        StructField("GEOID20", StringType(), True),
        StructField("ZCTA5CE20", StringType(), True),
        StructField("latitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("value", FloatType(), True),
        StructField("date", TimestampType(), True)
    ])
    
    # Broadcast the points GeoDataFrame to all executors
    broadcast_points = sc.broadcast(points_gdf)
    
    # Split files into batches
    batches = [bil_files[i:i + batch_size] for i in range(0, len(bil_files), batch_size)]
    
    result_df = spark.createDataFrame([], result_schema)
    
    # Process each batch and union results
    for i, batch in enumerate(batches):
        print(f"Processing batch {i+1}/{len(batches)}")
        
        # Process this batch in parallel across cluster
        batch_rdd = sc.parallelize(batch)
        batch_results = batch_rdd.mapPartitions(
            lambda files: process_files_partition(files, broadcast_points.value)
        ).collect()
        
        # Convert batch results to DataFrame and union with main result
        if batch_results:
            batch_df = spark.createDataFrame(batch_results, result_schema)
            result_df = result_df.unionByName(batch_df)
        
            if (i+1) % 5 == 0:
                temp_path = f"/FileStore/intermediate_output/temp_batch_{i}"
                result_df.write.mode("overwrite").parquet(temp_path)
                result_df = spark.read.parquet(temp_path)
    
    return result_df


def process_files_partition(files_iterator, points_gdf):
    """
    Process files within a partition
    """
    results = []
    for file_path in files_iterator:
        try:
            # Process single file and get results
            file_results = process_single_file(file_path, points_gdf)
            results.extend(file_results)
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
    return results


def process_single_file(file_path, points_gdf):
    """
    Process a single file and return row dictionaries
    """
    # Extract date from filename
    date_match = re.search(r'(\d{4})(\d{2})(\d{2})', os.path.basename(file_path))
    if date_match:
        year, month, day = map(int, date_match.groups())
        date = pd.to_datetime(f"{year}-{month}-{day}")
    else:
        mod_time = os.path.getmtime(file_path)
        date = pd.to_datetime(datetime.datetime.fromtimestamp(mod_time))
    
    # Read BIL file
    data, metadata = read_bil_file(file_path)
    if data is None or metadata is None:
        return []
    
    # Extract values at points
    values = get_values_at_points(points_gdf, data, metadata)
    
    # Create result dictionaries
    results = []
    for idx, point in points_gdf.iterrows():
        result = {col: point[col] for col in points_gdf.columns if col != 'geometry'}
        result['value'] = values[idx]
        result['date'] = date
        results.append(result)
    
    return results

def process_batch(bil_files_batch, points_gdf):
    """
    Process a batch of BIL files 
    """
    # Create a list for batch results
    batch_dfs = []
    
    # Process each BIL file
    for bil_file in bil_files_batch:
        file_name = os.path.basename(bil_file)
        column_name = os.path.splitext(file_name)[0]
        
        print(f"Processing {file_name}...")
        
        # Read BIL file
        bil_data, bil_metadata = read_bil_file(bil_file)
        
        if bil_data is None or bil_metadata is None:
            print(f"Skipping {file_name} due to reading errors")
            continue
        
        # Extract values at zip code points
        values = get_values_at_points(points_gdf, bil_data, bil_metadata)
        
        # Create a list of dictionaries for this file's results
        file_results = []
        for idx, point in points_gdf.iterrows():
            result = {}
            # Add only non-geometry columns from points_gdf
            for col in points_gdf.columns:
                if col != 'geometry':
                    result[col] = point[col]
            
            # Add coordinates
            result['longitude'] = point.geometry.x
            result['latitude'] = point.geometry.y
            
            # Add value for this file
            result[column_name] = float(values[idx]) if not pd.isna(values[idx]) else None
            
            file_results.append(result)
        
        # Create DataFrame for this file
        if file_results:
            file_df = spark.createDataFrame(file_results)
            batch_dfs.append(file_df)
    
    return batch_dfs

## Location Data Ingestion

In [0]:
# DBFS path
data_path = "/dbfs/FileStore/tables/"

# Load zip code boundaries
zip_gdf = load_zip_shapefile()

Loaded 33791 zip code boundaries


In [0]:
zip_state_df_path = "/FileStore/tables/uscities.csv"
zip_state_df = spark.read.csv(zip_state_df_path, header=True, inferSchema=True)

In [0]:
print("Calculating zip code representative points...")
points_gdf = get_zip_points(zip_gdf)

Calculating zip code representative points...


## Precipitation Data Ingestion and Processing

In [0]:
batch_size = 10
data_path = "/FileStore/tables/"
all_bil_files = list_bil_files(data_path)
sc = spark.sparkContext

### 2023 Data 

In [0]:
# Filter to year 2023 files
bil_files = [f for f in all_bil_files if "2023" in f]
print(f"Found {len(bil_files)} BIL files for 2023")

Found 365 BIL files for 2023


In [0]:
# Split into batches
batches = [bil_files[i:i + batch_size] for i in range(0, len(bil_files), batch_size)]

In [0]:
# Define common columns for joining
common_cols = [
    'ALAND20', 'AWATER20', 'CLASSFP20', 'FUNCSTAT20', 'GEOID20', 
    'INTPTLAT20', 'INTPTLON20', 'MTFCC20', 'ZCTA5CE20', 
    'latitude', 'longitude'
]

# Broadcast the points GeoDataFrame
broadcast_points = sc.broadcast(points_gdf)

In [0]:
points_dict = points_gdf.to_dict('records')
geometries = {i: {'x': point.geometry.x, 'y': point.geometry.y} 
                for i, point in enumerate(points_gdf.itertuples())}

In [0]:
result_df = None

In [0]:
for i, batch in enumerate(batches):
    print(f"Processing batch {i+1}/{len(batches)}")
    
    # Process this batch
    batch_dfs = process_batch(batch, points_gdf)
    
    if not batch_dfs:
        continue
    
    # Join all DataFrames in this batch
    batch_result = reduce(lambda df1, df2: df1.join(df2, common_cols, "left"), batch_dfs)
    
    # If this is the first batch, initialize result_df
    if result_df is None:
        result_df = batch_result
    else:
        # Join with previous batches
        result_df = result_df.join(batch_result, common_cols, "left")
        
        # Checkpoint to disk periodically
        if (i+1) % 5 == 0 or i == len(batches)-1:
            checkpoint_path = f"/FileStore/intermediate_output/checkpoint_{i}"
            result_df.write.mode("overwrite").parquet(checkpoint_path)
            result_df = spark.read.parquet(checkpoint_path)

# Write final result
if result_df is not None:
    result_df.write.mode("overwrite").parquet("/FileStore/intermediate_output/2025_precipitation_data")
    print(f"Processing complete. Final dataset has {result_df.count()} rows.")
else:
    print("No results were produced.")

Processing batch 1/37
Processing PRISM_ppt_stable_4kmD2_20230101_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230102_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230103_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230104_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230105_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230106_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230107_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230108_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230109_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230110_bil.bil...
Processing batch 2/37
Processing PRISM_ppt_stable_4kmD2_20230111_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230112_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230113_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230114_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230115_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230116_bil.bil...
Processing PRISM_ppt_stable_4kmD2_20230117_bil.bil...
Processing PRISM_ppt_stable_4kmD2_2023

In [0]:
# Clear memory and decache everything
spark.catalog.clearCache()
gc.collect()

65228

## 2024 Data

In [0]:
data_path = "/FileStore/tables/"
all_bil_files = list_bil_files(data_path)

# Filter to year 2025 files
bil_files = [f for f in all_bil_files if "2024" in f]
print(f"Found {len(bil_files)} BIL files for 2024")

Found 366 BIL files for 2024


In [0]:
batches = [bil_files[i:i + batch_size] for i in range(0, len(bil_files), batch_size)]

In [0]:
for i, batch in enumerate(batches):
    print(f"Processing batch {i+1}/{len(batches)}")
    
    # Process this batch
    batch_dfs = process_batch(batch, points_gdf)
    
    if not batch_dfs:
        continue
    
    # Join all DataFrames in this batch
    batch_result = reduce(lambda df1, df2: df1.join(df2, common_cols, "left"), batch_dfs)
    
    # If this is the first batch, initialize result_df
    if result_df is None:
        result_df = batch_result
    else:
        # Join with previous batches
        result_df = result_df.join(batch_result, common_cols, "left")
        
        # Checkpoint to disk periodically
        if (i+1) % 5 == 0 or i == len(batches)-1:
            checkpoint_path = f"/FileStore/intermediate_output/checkpoint_{i}"
            result_df.write.mode("overwrite").parquet(checkpoint_path)
            result_df = spark.read.parquet(checkpoint_path)

# Write final result
if result_df is not None:
    result_df.write.mode("overwrite").parquet("/FileStore/intermediate_output/2024_precipitation_data")
    print(f"Processing complete. Final dataset has {result_df.count()} rows.")
else:
    print("No results were produced.")

Processing batch 1/37
Processing PRISM_ppt_provisional_4kmD2_20240901_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240902_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240903_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240904_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240905_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240906_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240907_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240908_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240909_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240910_bil.bil...
Processing batch 2/37
Processing PRISM_ppt_provisional_4kmD2_20240911_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240912_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240913_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240914_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240915_bil.bil...
Processing PRISM_ppt_provisional_4kmD2_20240916_bil.bil...
Processing P

## 2025 data 

In [0]:
data_path = "/FileStore/tables/"
all_bil_files = list_bil_files(data_path)

# Filter to year 2025 files
bil_files = [f for f in all_bil_files if "2025" in f]
print(f"Found {len(bil_files)} BIL files for 2025")

batches = [bil_files[i:i + batch_size] for i in range(0, len(bil_files), batch_size)]

Found 78 BIL files for 2025


In [0]:
for i, batch in enumerate(batches):
    print(f"Processing batch {i+1}/{len(batches)}")
    
    # Process this batch
    batch_dfs = process_batch(batch, points_gdf)
    
    if not batch_dfs:
        continue
    
    # Join all DataFrames in this batch
    batch_result = reduce(lambda df1, df2: df1.join(df2, common_cols, "left"), batch_dfs)
    
    # If this is the first batch, initialize result_df
    if result_df is None:
        result_df = batch_result
    else:
        # Join with previous batches
        result_df = result_df.join(batch_result, common_cols, "left")
        
        # Checkpoint to disk periodically
        if (i+1) % 5 == 0 or i == len(batches)-1:
            checkpoint_path = f"/FileStore/intermediate_output/checkpoint_{i}"
            result_df.write.mode("overwrite").parquet(checkpoint_path)
            result_df = spark.read.parquet(checkpoint_path)

# Write final result
if result_df is not None:
    result_df.write.mode("overwrite").parquet("/FileStore/intermediate_output/2025_precipitation_data")
    print(f"Processing complete. Final dataset has {result_df.count()} rows.")
else:
    print("No results were produced.")

Processing batch 1/8
Processing PRISM_ppt_early_4kmD2_20250301_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250302_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250303_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250304_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250305_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250306_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250307_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250308_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250309_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250310_bil.bil...
Processing batch 2/8
Processing PRISM_ppt_early_4kmD2_20250311_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250312_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250313_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250314_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250315_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250316_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250317_bil.bil...
Processing PRISM_ppt_early_4kmD2_20250318_bil.bil...
Proc

In [0]:
spark.stop()